In [13]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import gzip
import time
import shutil
import requests



In [62]:
def download():
  
    i = requests.get('http://data.dot.state.mn.us/iris_xml/incident.xml.gz')
    with open('incidents.xml', 'w') as handle:
        handle.write(gzip.decompress(i.content).decode('utf-8'))
    d = requests.get('http://data.dot.state.mn.us/iris_xml/det_sample.xml.gz')
    with open('det_sample.xml', 'w') as handle:
        handle.write(gzip.decompress(d.content).decode('ISO-8859-1'))
    s = requests.get('http://data.dot.state.mn.us/iris_xml/stat_sample.xml.gz')
    with open('stat_sample.xml', 'w') as handle:
        handle.write(gzip.decompress(s.content).decode('utf-8'))

In [111]:
def data_check():
        XMLfile = "incident.xml"
        try:
            with open('crash_data.csv', 'r') as CD:
                incidents()
        except FileNotFoundError:
                All_Crash_Data = pd.DataFrame(columns=['Name', 'Date', 'Direction', 'Road', 'Location', 'Event'])
                with open('crash_data.csv', 'w') as f:
                    All_Crash_Data.to_csv(f, header=True)
                    incidents()
        XMLfile = "det_sample.xml"
        try:
            with open('detector_data.csv', 'r') as CD:
                detectors()
        except FileNotFoundError:
                Detector_Data = pd.DataFrame(columns=['Sensor', 'Time', 'Occupancy', 'Speed', 'Flow'])
                with open('detector_data.csv', 'w') as f:
                    Detector_Data.to_csv(f, header=True)
                    detectors()
        XMLfile = "stat_sample.xml"
        try:
            with open('station_data.csv', 'r') as CD:
                stations()
        except FileNotFoundError:
                Station_Data = pd.DataFrame(columns=['Label', 'Time', 'Detectors', 'Lat', 'Lng'])
                with open('station_data.csv', 'w') as f:
                    Station_Data.to_csv(f, header=True)
                    stations()

In [77]:
def incidents():
        XMLfile = "incidents.xml"


        dates = []
        incident_dirs = []
        roads = []
        locations = []
        names = []
        events = []

        parsedXML = ET.parse(XMLfile)
        root = parsedXML.getroot()
        for child in root:
            try:
                dates.append(child.attrib['event_date'])
            except KeyError:
                dates.append("none")
            try:
                names.append(str(child.attrib['name']))
            except KeyError:
                name.append("none")
            try:
                incident_dirs.append(child.attrib['dir'])
            except KeyError:
                incident_dir.append("none")
            try:
                roads.append(child.attrib['road'])
            except KeyError:
                roads.append('None')
            try:
                locations.append(child.attrib['location'])
            except KeyError:
                locations.append("none")
            try: 
                event = child.attrib['event_type'].split("_", 1)
                events.append(event[1])
            except KeyError:
                events.append("none")



        DF = pd.DataFrame({"Name" : names,
                           "Date" : dates,
                           "Direction": incident_dirs,
                           "Road" : roads,
                           "Location" : locations,
                           "Event" : events})

        print(DF)


        with open('crash_data.csv', 'a') as f:
            DF.to_csv(f, header=False)


In [105]:
def detectors():
        XMLfile = "det_sample.xml"

        cooridors = []
        times = []
        detectors = []
        lats = []
        lngs = []

        parsedXML = ET.parse(XMLfile)
        root = parsedXML.getroot()
#         print(root.findall("./time_stamp"))
        for child in root:
            try:
                cooridors.append(child.attrib['label'])
            except KeyError:
                cooridors.append("none")
#             try:
#                 times.append(str(child.attrib['tms_config_time_stamp']))
#             except KeyError:
#                 times.append("none")
            try:
                detectors.append(child.attrib['detector name'])
            except KeyError:
                detectors.append("none")
            try:
                lats.append(child.attrib['lat'])
            except KeyError:
                lats.append('None')
            try:
                lngs.append(child.attrib['lon'])
            except KeyError:
                lngs.append("none")



        DF = pd.DataFrame({"Label" : cooridors,
                           "Sensor" : detectors,
#                            "Time" : times,
                           "Lat": lats,
                           "Lng" : lngs})

        print(DF)


        with open('station_data.csv', 'a') as f:
            DF.to_csv(f, header=False)


In [107]:
def stations():
        XMLfile = "stat_sample.xml"

        sensors = []
        times = []
        flows = []
        occupancies = []
        speeds = []

        parsedXML = ET.parse(XMLfile)
        root = parsedXML.getroot()
#         print(root.findall("./time_stamp"))
        for child in root:
            try:
                sensors.append(child.attrib['sensor'])
            except KeyError:
                sensors.append("none")
#             try:
#                 times.append(str(child.attrib['time_stamp']))
#             except KeyError:
#                 times.append("none")
            try:
                flows.append(child.attrib['sample flow'])
            except KeyError:
                flows.append("none")
            try:
                occupancies.append(child.attrib['occ'])
            except KeyError:
                occupancies.append('None')
            try:
                speeds.append(child.attrib['speed'])
            except KeyError:
                speeds.append("none")



        DF = pd.DataFrame({"Sensor" : sensors,
#                            "Time" : times,
                           "Occupancy": occupancies,
                           "Speed" : speeds,
                           "Flow" : flows})

        print(DF)


        with open('detector_data.csv', 'a') as f:
            DF.to_csv(f, header=False)

In [108]:
detectors()

     Label Sensor   Lat   Lng
0     none   none  None  none
1     none   none  None  none
2     none   none  None  none
3     none   none  None  none
4     none   none  None  none
5     none   none  None  none
6     none   none  None  none
7     none   none  None  none
8     none   none  None  none
9     none   none  None  none
10    none   none  None  none
11    none   none  None  none
12    none   none  None  none
13    none   none  None  none
14    none   none  None  none
15    none   none  None  none
16    none   none  None  none
17    none   none  None  none
18    none   none  None  none
19    none   none  None  none
20    none   none  None  none
21    none   none  None  none
22    none   none  None  none
23    none   none  None  none
24    none   none  None  none
25    none   none  None  none
26    none   none  None  none
27    none   none  None  none
28    none   none  None  none
29    none   none  None  none
...    ...    ...   ...   ...
6652  none   none  None  none
6653  none

In [112]:
download()

In [113]:
data_check()

                Name                          Date Direction     Road  \
0       L004_9942278  Sun Sep 30 16:57:42 CDT 2018        SB   T.H.61   
1   2018093016521761  Sun Sep 30 16:52:17 CDT 2018        WB    I-494   
2   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB   T.H.62   
3       L004_9942259  Sun Sep 30 16:50:52 CDT 2018        SB   T.H.52   
4   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB  U.S.169   
5   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB  U.S.169   
6   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB  T.H.100   
7   2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB    I-35W   
8   2018092820040732  Fri Sep 28 20:04:07 CDT 2018        SB    I-35E   
9   2018092820090159  Fri Sep 28 20:09:01 CDT 2018        WB   T.H.36   
10  2018092819042630  Fri Sep 28 19:04:26 CDT 2018        SB  T.H.100   

            Location     Event  
0        @ T.H.36 EB     STALL  
1   @ Excelsior Blvd    HAZARD  
2          @ T.H.100  RO